In [7]:
import itertools
import numpy as np
import requests
import pandas as pd
import json
import os

### Slack identity

In [8]:
!pip install slackclient
from slackclient import SlackClient

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
slack_oauth_token = os.environ["SLACK_TOKEN"]

In [11]:
sc = SlackClient(slack_oauth_token)

In [12]:
channels = sc.api_call("conversations.list")['channels']


In [13]:
# Get shared-channel ids
shared_channels = ['big-d-and-the-v']
kickerscore_channel_ids = []
for d in channels:
    if d['name'] in shared_channels:
        kickerscore_channel_ids.append(d['id'])
        

In [14]:
# Get member ids
current_slack_members = []
import logging
logger = logging.getLogger(__name__)
for channel_id in kickerscore_channel_ids:
    logger.info("Running new channel member check for {}".format(channel_id))
    current_slack_members += sc.api_call(
        "conversations.members", channel=channel_id)["members"]

In [15]:
# map member ids to names
id2name = {}
for pl in current_slack_members:
    player_info = sc.api_call("users.info", user=pl)["user"]
    id2name[pl] = player_info['profile']['real_name']

In [16]:
id2name

{'U23UM3VA7': 'Bertjan Broeksema',
 'U240800MP': 'Steven Reitsma',
 'U2408BZE2': 'Hans Adriaans',
 'U2409HL72': 'Coen Seinen',
 'U2409KS1E': 'Alexander Backus',
 'U240SPC5B': 'Gerben Oostra',
 'U276J8Z4Z': 'Sophron Vermeij',
 'U2XCYG1S8': 'Michel Meulpolder',
 'U39UDRUUX': 'Bram Neijt',
 'U3L16HPBP': 'Davey Mollinger',
 'U3LJ5CX0C': 'Paula Amaral Santos',
 'U3ZFYF9U5': 'Ruurtjan Pul',
 'U4SRCTC0H': 'Robbert van der Gugten',
 'U567PJY3T': 'Roman Daukuls',
 'U5MTUMEQP': 'Tom Lemmens',
 'U62H5E89X': 'Tom de Ruijter',
 'U6HD4QUA3': 'Dennis Ramondt',
 'U8MH0DSER': 'Erik Jan de Vries',
 'U8NS8SF7A': 'Edwin de Jong',
 'U8QNBGFQC': 'Fred',
 'U9AKYPJNL': 'Diederik Jan Lemkes',
 'U9CNB16TW': 'Rob Brinkman',
 'U9T1TAYTG': 'Jaap de Koning',
 'UAGCD18TX': 'Stefan Verbruggen',
 'UBGSMHPL1': 'Axel Goblet',
 'UBHGYUV8C': 'Sven Stringer',
 'UBJ71CQ9G': 'Burcak',
 'UCUQG916E': 'Huib Keemink',
 'UD4HYDTL2': 'Gerard Toonstra',
 'UFASRK4MD': 'Daphne Hensbergen',
 'UFPC1NXEH': 'Wijnand',
 'UBH8BU2CT': 'Jasp

## Check the flask app

In [17]:
d = {'slack_ids' :  ["UDTKJ3DPA", "UBGSMHPL1"]} # Mike & Axel
d = {'slack_ids' :  ["U3L16HPBP"]} # Davey
# d = {'slack_ids' :  ['UBH8BU2CT']} # Jasper D.
# d = {'slack_ids' :  ['U6HD4QUA3']} # Dennis --> Doesn't exist as a key
# d = {'slack_ids' :  ['UBH8BU2CT', 'U6HD4QUA3']} # Jasper D. & Dennis
ids = requests.post("http://localhost:8080", headers={'Content-Type': 'application/json'}, data=json.dumps(d))
for ix, i in enumerate(json.loads(ids.text)['slack_ids']):
    try:
        print("#%i: %s"%(ix+1, id2name[i]))
    except KeyError:
        print("#%i: -"%(ix+1))

#1: Robbert van der Gugten
#2: Axel Goblet
#3: Richard Bartels
#4: Huib Keemink
#5: Ruurtjan Pul
